In [1]:
#| echo: true

from examples.example_2_branching_multistep.ex_2_model_classes import Trial, g
import pandas as pd
import os

In [2]:
#| echo: true

g.sim_duration = 3000
g.number_of_runs = 1

In [3]:
#| echo: false
#| output: asis
# Path to the external Python script
file_path = "../examples/example_2_branching_multistep/ex_2_model_classes.py"

# Read the file content
if os.path.exists(file_path):
    with open(file_path, "r") as f:
        code_content = f.read()
else:
    code_content = "File not found."
with open(file_path, "r") as f:
    code_content = f.read()

# Print the Quarto `{details}` block for collapsible output
print(f"""
:::{{.callout-note collapse="true"}}
### View Imported Code, which has had logging steps added at the appropriate points in the 'model' class

```python
{code_content}
```

:::

""")


:::{.callout-note collapse="true"}
### View Imported Code, which has had logging steps added at the appropriate points in the 'model' class

```python
import random
import numpy as np
import pandas as pd
import simpy
from sim_tools.distributions import Exponential, Lognormal, Uniform, Normal, Bernoulli
from vidigi.utils import populate_store
from examples.simulation_utility_functions import trace


# Class to store global parameter values.  We don't create an instance of this
# class - we just refer to the class blueprint itself to access the numbers
# inside.
class g:
    '''
    Create a scenario to parameterise the simulation model

    Parameters:
    -----------
    random_number_set: int, optional (default=DEFAULT_RNG_SET)
        Set to control the initial seeds of each stream of pseudo
        random numbers used in the model.

    n_triage: int
        The number of triage cubicles

    n_reg: int
        The number of registration clerks

    n_exam: int
        The number o

In [4]:
#| echo: true

my_trial = Trial()

my_trial.run_trial()

In [5]:
#| echo: true
my_trial.all_event_logs.head(10)

,patient,pathway,event,event_type,time,resource_id,run
0,1,Shared,arrival,arrival_departure,3.285355,NaN,0
1,1,Non-Trauma,triage_wait_begins,queue,3.285355,NaN,0
2,1,Non-Trauma,triage_begins,resource_use,3.285355,1.0,0
3,2,Shared,arrival,arrival_departure,3.289691,NaN,0
4,2,Non-Trauma,triage_wait_begins,queue,3.289691,NaN,0
5,2,Non-Trauma,triage_begins,resource_use,3.289691,2.0,0
6,1,Non-Trauma,triage_complete,resource_use_end,7.364946,1.0,0
7,1,Non-Trauma,MINORS_registration_wait_begins,queue,7.364946,NaN,0
8,1,Non-Trauma,MINORS_registration_begins,resource_use,7.364946,1.0,0
9,2,Non-Trauma,triage_complete,resource_use_end,9.407274,2.0,0


In [6]:
#| echo: true

my_trial.all_event_logs.event_type.value_counts()

event_type
queue                 2100
resource_use          2035
resource_use_end      2021
arrival_departure     1167
attribute_assigned     264
Name: count, dtype: int64

In [7]:
#| echo: true

my_trial.all_event_logs.event.value_counts()

event
arrival                             623
triage_wait_begins                  623
triage_begins                       622
triage_complete                     620
depart                              544
MINORS_registration_wait_begins     539
MINORS_registration_begins          514
MINORS_registration_complete        512
MINORS_examination_wait_begins      512
MINORS_examination_begins           475
MINORS_examination_complete         472
requires_treatment                  264
MINORS_treatment_wait_begins        264
MINORS_treatment_begins             264
MINORS_treatment_complete           262
TRAUMA_stabilisation_wait_begins     81
TRAUMA_stabilisation_begins          81
TRAUMA_stabilisation_complete        81
TRAUMA_treatment_wait_begins         81
TRAUMA_treatment_begins              79
TRAUMA_treatment_complete            74
Name: count, dtype: int64

For simplicity, we'll only include those patients who actually went through the whole system. 

Note - this could be very misleading in systems with huge capacity issues!

In [8]:
#| echo: true

# First, identify all patients who have a 'depart' event
patients_with_depart = my_trial.all_event_logs[my_trial.all_event_logs['event'].str.contains('depart')]['patient'].unique()

# Then filter the original DataFrame to only include those patients
filtered_df = my_trial.all_event_logs[my_trial.all_event_logs['patient'].isin(patients_with_depart)]

logs_transformed = filtered_df[~filtered_df['event'].str.contains('wait')].copy()
logs_transformed = logs_transformed[logs_transformed['event_type'].isin(['resource_use', 'resource_use_end'])].copy()
logs_transformed['event_stage'] = logs_transformed['event_type'].apply(lambda x: 'complete' if 'end' in x else 'start')
logs_transformed['event_name'] = logs_transformed['event'].str.replace('_begins|_complete', '', regex=True)
logs_transformed['resource_id_full'] = logs_transformed.apply(lambda x: f"{x['event_name']}_{x['resource_id']:.0f}", axis=1)
logs_transformed = logs_transformed.sort_values(['run', 'time'], ascending=True)
# logs_transformed["activity_id"] = (
#     logs_transformed.groupby(["run", "patient", "event_name"]).ngroup() + 1
# )

# logs_transformed = logs_transformed.sort_values(["run", "patient", "activity_id", "event_stage"], ascending=[True, True, True, False])

# Sort the data by run, patient, time, and event_name to handle tied start times
logs_transformed = logs_transformed.sort_values(["run", "patient", "time", "event_name"])

# Get the first occurrence of each activity (the start event)
first_occurrences = (
    logs_transformed[logs_transformed["event_stage"] == "start"]
    .drop_duplicates(["run", "patient", "event_name"])
    .copy()
)

# Sort by time within each run to determine the proper sequence
first_occurrences = first_occurrences.sort_values(["run", "time", "event_name"])

# Assign sequential activity_id within each run
first_occurrences["activity_id"] = first_occurrences.groupby("run").cumcount() + 1

# Merge the activity_id back to the main DataFrame
logs_transformed = logs_transformed.merge(
    first_occurrences[["run", "patient", "event_name", "activity_id"]],
    on=["run", "patient", "event_name"],
    how="left"
)

# Sort for final ordering
logs_transformed = logs_transformed.sort_values(
    ["run", "patient", "activity_id", "event_stage"],
    ascending=[True, True, True, False]
)
logs_transformed.head(50)

,patient,pathway,event,event_type,time,resource_id,run,event_stage,event_name,resource_id_full,activity_id
0,1,Non-Trauma,triage_begins,resource_use,3.285355,1.0,0,start,triage,triage_1,1
2,1,Non-Trauma,triage_complete,resource_use_end,7.364946,1.0,0,complete,triage,triage_1,1
1,1,Non-Trauma,MINORS_registration_begins,resource_use,7.364946,1.0,0,start,MINORS_registration,MINORS_registration_1,3
4,1,Non-Trauma,MINORS_registration_complete,resource_use_end,15.418481,1.0,0,complete,MINORS_registration,MINORS_registration_1,3
3,1,Non-Trauma,MINORS_examination_begins,resource_use,15.418481,1.0,0,start,MINORS_examination,MINORS_examination_1,5
5,1,Non-Trauma,MINORS_examination_complete,resource_use_end,31.636252,1.0,0,complete,MINORS_examination,MINORS_examination_1,5
6,2,Non-Trauma,triage_begins,resource_use,3.289691,2.0,0,start,triage,triage_2,2
8,2,Non-Trauma,triage_complete,resource_use_end,9.407274,2.0,0,complete,triage,triage_2,2
7,2,Non-Trauma,MINORS_registration_begins,resource_use,9.407274,2.0,0,start,MINORS_registration,MINORS_registration_2,4
10,2,Non-Trauma,MINORS_registration_complete,resource_use_end,17.104670,2.0,0,complete,MINORS_registration,MINORS_registration_2,4


In [9]:
#| echo: true

logs_transformed[(logs_transformed["run"]==1) & (logs_transformed["activity_id"]==26)]

,patient,pathway,event,event_type,time,resource_id,run,event_stage,event_name,resource_id_full,activity_id


In [10]:
#| echo: true

logs_transformed[logs_transformed["activity_id"]==26].sort_values('run').head(30)

,patient,pathway,event,event_type,time,resource_id,run,event_stage,event_name,resource_id_full,activity_id
60,10,Trauma,triage_begins,resource_use,129.468902,2.0,0,start,triage,triage_2,26
62,10,Trauma,triage_complete,resource_use_end,165.815421,2.0,0,complete,triage,triage_2,26


In [11]:
#| echo: true
logs_transformed.sort_values('activity_id').head(20)

,patient,pathway,event,event_type,time,resource_id,run,event_stage,event_name,resource_id_full,activity_id
0,1,Non-Trauma,triage_begins,resource_use,3.285355,1.0,0,start,triage,triage_1,1
2,1,Non-Trauma,triage_complete,resource_use_end,7.364946,1.0,0,complete,triage,triage_1,1
6,2,Non-Trauma,triage_begins,resource_use,3.289691,2.0,0,start,triage,triage_2,2
8,2,Non-Trauma,triage_complete,resource_use_end,9.407274,2.0,0,complete,triage,triage_2,2
1,1,Non-Trauma,MINORS_registration_begins,resource_use,7.364946,1.0,0,start,MINORS_registration,MINORS_registration_1,3
4,1,Non-Trauma,MINORS_registration_complete,resource_use_end,15.418481,1.0,0,complete,MINORS_registration,MINORS_registration_1,3
7,2,Non-Trauma,MINORS_registration_begins,resource_use,9.407274,2.0,0,start,MINORS_registration,MINORS_registration_2,4
10,2,Non-Trauma,MINORS_registration_complete,resource_use_end,17.104670,2.0,0,complete,MINORS_registration,MINORS_registration_2,4
3,1,Non-Trauma,MINORS_examination_begins,resource_use,15.418481,1.0,0,start,MINORS_examination,MINORS_examination_1,5
5,1,Non-Trauma,MINORS_examination_complete,resource_use_end,31.636252,1.0,0,complete,MINORS_examination,MINORS_examination_1,5


In [12]:
#| echo: true
logs_transformed[["event_name", "event_stage", "event_type"]].value_counts()

event_name            event_stage  event_type      
triage                complete     resource_use_end    544
                      start        resource_use        544
MINORS_examination    complete     resource_use_end    470
                      start        resource_use        470
MINORS_registration   complete     resource_use_end    470
                      start        resource_use        470
MINORS_treatment      complete     resource_use_end    262
                      start        resource_use        262
TRAUMA_stabilisation  complete     resource_use_end     74
                      start        resource_use         74
TRAUMA_treatment      complete     resource_use_end     74
                      start        resource_use         74
Name: count, dtype: int64

In [13]:
logs_transformed.event.value_counts()

event
triage_begins                    544
triage_complete                  544
MINORS_registration_begins       470
MINORS_registration_complete     470
MINORS_examination_begins        470
MINORS_examination_complete      470
MINORS_treatment_begins          262
MINORS_treatment_complete        262
TRAUMA_stabilisation_begins       74
TRAUMA_stabilisation_complete     74
TRAUMA_treatment_begins           74
TRAUMA_treatment_complete         74
Name: count, dtype: int64

For ease, now let's save these results as a file that we can load into R. 

We could use a csv for easy interoperability. Alternatively, we could use something like Feather or Parquet, which are usable by both R and Python while retaining data types.

For ease of use and long-term readbility, we will use csv in this case. 

In [14]:
logs_transformed.to_csv('simulation_logs_for_bupar.csv', index=False)